<a href="https://colab.research.google.com/github/bgtripp/Computational-Modeling-and-Simulation/blob/master/Textgenrnn_ML_Data_Journalism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #Helpful for importing and manipulating data
import numpy as np #Helpful for working with arrays of data and math
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

#https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb

url = 'https://raw.githubusercontent.com/walkerkq/musiclyrics/master/billboard_lyrics_1964-2015.csv'

song_data = pd.read_csv(url, encoding = "latin-1") #Pandas dataframe
song_data.head() #Shows the first 5 lines of the dataframe

Using TensorFlow backend.


,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [2]:
year = 1965

subset = song_data[song_data['Year'] == year]
lyrics = subset['Lyrics']
lyrics.to_csv(r'lyrics.txt', header=None, index=None, sep=' ', mode='a')
file_name = 'lyrics.txt'
model_name = '{}PopLyrics'.format(year)

model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 30,   # set higher to train the model for longer default 20
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

lyrics.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


0    sam the sham miscellaneous wooly bully wooly b...
1     sugar pie honey bunch you know that i love yo...
2                                                     
3     when i woke up this morning you were on my mi...
4     you never close your eyes anymore when i kiss...
Name: Lyrics, dtype: object

In [0]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])












99 texts collected.
Training new model w/ 3-layer, 128-cell LSTMs
Training on 68,873 character sequences.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
67/67 [==============================] - 149s 2s/step - loss: 2.9753
Epoch 2/30
67/67 [==============================] - 145s 2s/step - loss: 2.8288
Epoch 3/30
67/67 [==============================] - 145s 2s/step - loss: 2.8120
Epoch 4/30
67/67 [==============================] - 143s 2s/step - loss: 2.5834
Epoch 5/30
67/67 [==============================] - 143s 2s/step - loss: 2.0729
####################
Temperature: 0.2
####################
" wo me love te lee eve  feer er te to  me lee te te le el te te le te  roe te re ere se le a me no we le le lo go no  foe ne te ere to  me you ne te te ere  love ne te te ere yo  be e ne te te lee se te  an to me yo me   mee me te lee to me  ne yo  be be te re lee te le te  lee te lee te le eve  

" the a love ne te ere te a 

In [0]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

In [0]:
textgen = textgenrnn(weights_path='1965PopLyrics_weights.hdf5',
                      vocab_path='1965PopLyrics_vocab.json',
                      config_path='1965PopLyrics_config.json')

In [0]:
textgen.generate_samples()